In [ ]:
import math
import sys

sys.path.append("../")

import IPython
import matplotlib.pylab as plt
import numpy as np

from constants import SPEED_OF_SOUND

%reload_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import wave

fname = '../data/test/random'

with wave.open(f'{fname}.wav', 'r') as f:
    n = f.getnframes()
    fs = f.getframerate()
    channels = f.getnchannels()
    print(n, fs, channels)
    bytes_ = f.readframes(n)
    print('duration', len(bytes_) / channels / 4 / fs)
array = np.frombuffer(bytes_, dtype=np.int32).reshape(channels, n)
print(len(bytes_))
print(array.shape)

array = array.astype(np.float32) / (2**31 - 1)
print(np.min(array), np.max(array))

IPython.display.Audio(array, rate=fs, normalize=True)

In [ ]:
from scipy.io import wavfile
fs, array_scipy = wavfile.read(f'{fname}_scipy.wav')

IPython.display.Audio(array_scipy, rate=fs, normalize=True)

# Test new broadband deterministic signals

In [ ]:
from signals import generate_signal
from math import ceil

duration = 3
n_frequencies = 2048
max_phase_shift = 100

frequencies = np.fft.rfftfreq(n_frequencies, 1/fs)
starts = np.random.choice(max_phase_shift, size=n_frequencies)

duration_total = duration + max_phase_shift / fs
n_buffer = int(ceil(duration * fs))

signals = np.array([generate_signal(fs, duration_total, "mono", frequency_hz=f)[s:s+n_buffer] for s, f in zip(starts, frequencies)])
print(signals.shape)
signals = np.sum(signals, axis=0)
print(signals.shape)

plt.figure()
plt.plot(signals)

IPython.display.Audio(signals, rate=fs)

In [ ]:
signals_f = np.fft.rfft(signals[:n_frequencies])
plt.scatter(frequencies, np.abs(signals_f)/n_buffer*2)

In [ ]:
from signals import generate_signal_random, linear_increase
fs = 48000
duration_sec = 30
signal = generate_signal_random(fs, duration_sec)
signal = linear_increase(signal, -10, 50)

IPython.display.Audio(signal, rate=fs, normalize=True)